In [6]:
%load_ext lab_black
import pandas as pd
from IPython.display import JSON
import warnings
import ast
import json

warnings.filterwarnings("ignore")

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [3]:
df = pd.read_csv("world_cup_2022_catar.csv")
df.shape

(64, 25)

In [ ]:
df.head(1)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

### Manipulating events_list, lineup_home and lineup_away columns into new columns

#### lineup_home manipulation

In [7]:
df["lineup_home"] = df["lineup_home"].str.replace("'", '"')
df["lineup_home"] = df["lineup_home"].apply(json.loads)

# transforming lineup_home list of dictionaries into new columns
def extract_player_data(lineup):
    if not lineup or not isinstance(lineup, list):
        return None, None
    player_names = [
        player["player_name"] for player in lineup if isinstance(player, dict)
    ]
    player_numbers = [
        player["player_number"] for player in lineup if isinstance(player, dict)
    ]
    if len(player_names) != len(player_numbers):
        print(lineup)
        return None, None
    return (player_names, player_numbers)


df["player_names_home"] = df.apply(
    lambda row: extract_player_data(row["lineup_home"])[0], axis=1
)
df["player_numbers_home"] = df.apply(
    lambda row: extract_player_data(row["lineup_home"])[1], axis=1
)

# retirar o espaçamento entre os valores
df["player_names_home"] = df["player_names_home"].apply(
    lambda x: list(map(str.strip, x))
)
df["player_numbers_home"] = df["player_numbers_home"].apply(
    lambda x: list(map(str.strip, x))
)

#### lineup_away manipulation

In [8]:
df["lineup_away"] = df["lineup_away"].str.replace("'", '"')
df["lineup_away"] = df["lineup_away"].apply(json.loads)

# transforming lineup_away list of dictionaries into new columns
def extract_player_data(lineup):
    if not lineup or not isinstance(lineup, list):
        return None, None
    player_names = [
        player["player_name"] for player in lineup if isinstance(player, dict)
    ]
    player_numbers = [
        player["player_number"] for player in lineup if isinstance(player, dict)
    ]
    if len(player_names) != len(player_numbers):
        print(lineup)
        return None, None
    return (player_names, player_numbers)


df["player_names_away"] = df.apply(
    lambda row: extract_player_data(row["lineup_away"])[0], axis=1
)
df["player_numbers_away"] = df.apply(
    lambda row: extract_player_data(row["lineup_away"])[1], axis=1
)

# retirar o espaçamento entre os valores
df["player_names_away"] = df["player_names_away"].apply(
    lambda x: list(map(str.strip, x))
)
df["player_numbers_away"] = df["player_numbers_away"].apply(
    lambda x: list(map(str.strip, x))
)

#### events_list manipulation

In [9]:
# getting the values from "events_list" columns and transforming to another dataframe.
df_events_list = pd.DataFrame(
    columns=[
        "team",
        "event_team",
        "event_time",
        "event_type",
        "action_player_1",
        "action_player_2",
    ]
)

for i, row in df.iterrows():
    events_list = ast.literal_eval(row["events_list"])
    for event in events_list:
        if event.get("event_team") == "home":
            team = row.team_name_home
        else:
            team = row.team_name_away
        df_events_list = df_events_list.append(
            {
                "team": team,
                "event_team": event.get("event_team"),
                "event_time": event.get("event_time"),
                "event_type": event.get("event_type"),
                "action_player_1": event.get("action_player_1"),
                "action_player_2": event.get("action_player_2"),
            },
            ignore_index=True,
        )

In [10]:
df_events_list

,team,event_team,event_time,event_type,action_player_1,action_player_2
0,Argentina,home,23',Penalty,Lionel Messi,Penalty
1,Argentina,home,36',Goal,Ángel Di María,Alexis Mac Allister
2,France,away,41',Substitution,Randal Kolo Muani,Ousmane Dembélé
3,France,away,41',Substitution,Marcus Thuram,Olivier Giroud
4,Argentina,home,52',Yellow card,Enzo Fernández,None
...,...,...,...,...,...,...
1038,Qatar,home,72',Substitution,Mohammed Muntari,Almoez Ali
1039,Ecuador,away,77',Substitution,José Cifuentes,Enner Valencia
1040,Qatar,home,78',Yellow card,Akram Afif,None
1041,Ecuador,away,90',Substitution,Kevin Rodríguez,Michael Estrada


In [11]:
df_events_list.to_csv("datasets/df_events_list.csv", index=False)

#### Creating some statistics

In [12]:
df_global_statistics = (
    df_events_list.event_type.value_counts()
    .to_frame()
    .reset_index()
    .rename({"index": "event_type", "event_type": "total"}, axis=1)
)
df_global_statistics.to_csv("datasets/df_global_statistics.csv", index=False)
df_global_statistics

,event_type,total
0,Substitution,587
1,Yellow card,224
2,Goal,153
3,PK,41
4,Penalty,17
5,Disallowed goal,9
6,Missed penalty,6
7,Second yellow card,3
8,Own goal,2
9,Red card,1


In [13]:
df_players_goals = df_events_list[
    (df_events_list.event_type == "Goal")
    | (df_events_list.event_type == "Penalty")
    | (df_events_list.event_type == "Own goal")
]
df_players_goals.to_csv("datasets/df_players_goals.csv", index=False)
df_players_goals

,team,event_team,event_time,event_type,action_player_1,action_player_2
0,Argentina,home,23',Penalty,Lionel Messi,Penalty
1,Argentina,home,36',Goal,Ángel Di María,Alexis Mac Allister
9,France,away,80',Penalty,Kylian Mbappé,Penalty
10,France,away,81',Goal,Kylian Mbappé,Marcus Thuram
18,Argentina,home,108',Goal,Lionel Messi,None
...,...,...,...,...,...,...
1023,England,home,71',Goal,Marcus Rashford,Harry Kane
1026,England,home,90',Goal,Jack Grealish,Callum Wilson
1027,Iran,away,103',Penalty,Mehdi Taremi,Penalty
1030,Ecuador,away,16',Penalty,Enner Valencia,Penalty


In [14]:
df_players_goals.team.value_counts().to_frame().reset_index().rename(
    {"index": "team", "team": "total_goals"}, axis=1
)

,team,total_goals
0,France,16
1,Argentina,15
2,England,13
3,Portugal,12
4,Netherlands,10
5,Spain,9
6,Croatia,8
7,Brazil,8
8,Morocco,6
9,Germany,6


In [17]:
df_players_goals.to_csv("datasets/goals_per_team.csv", index=False)

In [16]:
# top scorer

df_players_goals.action_player_1.value_counts().to_frame().reset_index().rename(
    {"index": "player_name", "action_player_1": "total_goals"}, axis=1
)

,player_name,total_goals
0,Kylian Mbappé,8
1,Lionel Messi,7
2,Julián Álvarez,4
3,Olivier Giroud,4
4,Richarlison,3
...,...,...
113,Ao Tanaka,1
114,Hwang Hee-chan,1
115,Kim Young-gwon,1
116,Ricardo Horta,1


In [18]:
df_players_goals.to_csv("datasets/top_scores.csv", index=False)

In [20]:
# top assists
top_assists = (
    df_players_goals.action_player_2.value_counts()
    .to_frame()
    .reset_index()
    .rename({"index": "player_name", "action_player_2": "total_assists"}, axis=1)
    .drop(index=0)
    .reset_index(drop=True)
)
top_assists

,player_name,total_assists
0,Harry Kane,3
1,Ivan Perišić,3
2,Lionel Messi,3
3,Antoine Griezmann,3
4,Bruno Fernandes,3
...,...,...
89,David Raum,1
90,Kaoru Mitoma,1
91,Junya Ito,1
92,César Azpilicueta,1


In [21]:
top_assists.to_csv("datasets/top_assists.csv", index=False)